In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#---------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Plot_Box_sns
import GrubbsTest

In [ ]:
conn_outages = Utilities.get_eemsp_oracle_connection()
conn_aws = Utilities.get_athena_prod_aws_connection()

In [ ]:
fig_num = 0

# ---------------------------------------------------------------
# USING PYTHON BETWEEN TWO SQL CALLS
# ---------------------------------------------------------------

In [ ]:
start = time.time()
##############################################################################################
# df_outage
##############################################################################################
dates_of_outages = ['2020-10-12', '2017-09-16']
outg_rec_nbs = ['11947217', '11946640', '10143524']

sql_outage = (
"""
SELECT   DOV.CI_NB, DOV.CMI_NB, DOV.OUTG_REC_NB, DOV.OUTAGE_NB, DOV.DT_ON_TS, DOV.DT_OFF_TS, 
         DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24) AS DT_OFF_TS_FULL, DOV.STEP_DRTN_NB,
         EXTRACT(YEAR FROM DOV.DT_OFF_TS)  AS START_YEAR, 
         DOV.OPERATING_UNIT_ID, DOV1.OPRTG_UNT_NM, DOV.STATE_ABBR_TX, PRIM.OFF_TM, PRIM.REST_TM, PRIM.PREMISE_NB, 
         DOV.MJR_CAUSE_CD, DOV.MNR_CAUSE_CD, DOV5.MJR_CAUSE_NM, DOV5.MNR_CAUSE_NM, 
         DOV4.EQUIP_TYP_NM, DOV4.SHORT_NM AS SHORT_NM_EQP_TYP, 
         DOV3.DVC_TYP_NM, DOV3.SHORT_NM AS SHORT_NM_CLR_DEV
FROM     DOVSADM.DOVS_OUTAGE_FACT DOV
         LEFT OUTER JOIN DOVSADM.DOVS_MASTER_GEO_DIM DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
         LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_ATTRIBUTES_DIM DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
         LEFT OUTER JOIN DOVSADM.DOVS_CLEARING_DEVICE_DIM DOV3 ON DOV.DEVICE_CD=DOV3.DEVICE_CD
         LEFT OUTER JOIN DOVSADM.DOVS_EQUIPMENT_TYPES_DIM DOV4 ON DOV.EQUIPMENT_CD=DOV4.EQUIPMENT_CD
         LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_CAUSE_TYPES_DIM DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD
         LEFT OUTER JOIN DOVSADM.DOVS_PREMISE_DIM PRIM ON DOV.OUTG_REC_NB=PRIM.OUTG_REC_NB 
WHERE    DOV.DT_OFF_TS IN ({})
AND  DOV2.INTRPTN_TYP_CD = 'S'
AND  DOV2.CURR_REC_STAT_CD = 'A' 
AND  DOV.MJR_CAUSE_CD <> 'NI' 
AND  DOV.DEVICE_CD <> 85 
AND  DOV.STATE_ABBR_TX = 'OH'
AND  DOV.OUTG_REC_NB IN ({})
"""
).format(','.join(["'{}'".format(x) for x in dates_of_outages]), 
         ','.join(["'{}'".format(x) for x in outg_rec_nbs]))

print(sql_outage)

#--------------------
df_outage = pd.read_sql(sql_outage, conn_outages)
#--------------------
#TODO
search_time_window = 60 #minutes
search_time_window = datetime.timedelta(minutes=search_time_window)

# NOTE: search_time_window can also be a vector of values...
df_outage['search_time_window'] = search_time_window
df_outage['t_search_min'] = df_outage['DT_OFF_TS_FULL'] - df_outage['search_time_window']
df_outage['t_search_max'] = df_outage['DT_ON_TS'] + df_outage['search_time_window']

time_tot_half = time.time() - start
print(f'time_half = {time_tot_half}')
##############################################################################################
# df_end_dev_event
##############################################################################################
cols_of_interest_end_dev_event = [
    'issuertracking_id',    
    'serialnumber',
    'enddeviceeventtypeid',
    'valuesinterval',
    'aep_premise_nb',
    'reason',
    'event_type',
    'longitude',
    'latitude',
    'aep_opco',
    'aep_event_dt'    
]

cols_of_interest_met_prem = [
    'mfr_devc_ser_nbr',
    'longitude',
    'latitude',
    'state_cd', 
    'prem_nb',
    'srvc_pole_nb',
    'trsf_pole_nb',
    'latitude_nb',
    'longitude_nb',
    'station_nb', 
    'xfmr_nb'
]

unique_dates = sorted(set(df_outage['t_search_min'].dt.date.unique().tolist() + 
                          df_outage['t_search_max'].dt.date.unique().tolist()))

regexp_pattern = r'(\\d{4}-\\d{2}-\\d{2})T(\\d{2}:\\d{2}:\\d{2}).*'
partial_sql_ede = (
"""
WITH EDE as (
    SELECT {}
    FROM meter_events.end_device_event
    WHERE aep_opco = 'oh'
    AND aep_event_dt IN ({})
    AND (
        {{}}
    )
),"""
).format(','.join(cols_of_interest_end_dev_event),
         ','.join(["'{}'".format(x) for x in unique_dates]))
#-----
ede_datetime_ranges_string = ''
for i, (t_search_min_max, sub_df_outage) in enumerate(df_outage.groupby(['t_search_min', 't_search_max'])):
    str_to_fmt = "(aep_event_dt IN ({}) AND aep_premise_nb IN ({}) AND CAST(regexp_replace(valuesinterval, {}, '$1 $2') AS TIMESTAMP) BETWEEN {} AND {})"
    if i>0:
        str_to_fmt = "\n        OR"+str_to_fmt
    ede_datetime_ranges_string += str_to_fmt.format(
        ','.join(["'{}'".format(x) for x in sorted(set([x.date() for x in t_search_min_max]))]), 
        ','.join(["'{}'".format(x) for x in sub_df_outage['PREMISE_NB'].unique().tolist()]), 
        f"'{regexp_pattern}'", 
        f"'{t_search_min_max[0]}'",
        f"'{t_search_min_max[1]}'"
    )   
partial_sql_ede = partial_sql_ede.format(ede_datetime_ranges_string)
#---------------------------------------------------------------------------------
partial_sql_mp = (
"""
MP AS (
    SELECT {}
    FROM default.meter_premise
    WHERE state_cd IN ({})
    AND prem_nb IN ({})
)
"""
).format(','.join(cols_of_interest_met_prem), 
         ','.join(["'{}'".format(x) for x in df_outage['STATE_ABBR_TX'].unique().tolist()]), 
         ','.join(["'{}'".format(x) for x in df_outage['PREMISE_NB'].unique().tolist()]))
#---------------------------------------------------------------------------------
sql_end_dev_event = (
"""{}{}
SELECT {}
FROM EDE
INNER JOIN MP
ON EDE.serialnumber = MP.mfr_devc_ser_nbr
"""
).format(partial_sql_ede, 
         partial_sql_mp, 
         ','.join([f'EDE.{x} AS {x}' for x in cols_of_interest_end_dev_event] + [f'MP.{x} AS {x}' for x in cols_of_interest_met_prem]))
#---------------------------------------------------------------------------------
print(sql_end_dev_event)

#--------------------
df_end_dev_event = pd.read_sql(sql_end_dev_event, conn_aws)
df_end_dev_event = Utilities_df.remove_prepend_from_columns_in_df(df_end_dev_event)
#--------------------
df_end_dev_event['valuesinterval_tz_aware'] = pd.to_datetime(df_end_dev_event['valuesinterval'])
df_end_dev_event['valuesinterval_local'] = df_end_dev_event['valuesinterval_tz_aware'].dt.tz_localize(None)
df_end_dev_event['valuesinterval_utc'] = df_end_dev_event['valuesinterval_tz_aware'].dt.tz_convert(None)
#--------------------
df_end_dev_event, n_events_with_mult_matches = clm.match_events_in_df_to_outages(df_end_dev_event, df_outage, 
                                                                                      return_n_events_with_mult_matches=True)
df_end_dev_event = clm.set_all_outages_info_in_events_df(df_end_dev_event, df_outage)
#--------------------
time_tot_0 = time.time() - start
print(f'time_NEW = {time_tot_0}')

In [ ]:
df_end_dev_event

# ---------------------------------------------------------------
# USING PYTHON BETWEEN TWO SQL CALLS
# MIMIC SQL ONLY
# ---------------------------------------------------------------

In [ ]:
start = time.time()
lowercase_cols_default = True
##############################################################################################
# df_outage_2
##############################################################################################
dates_of_outages = ['2020-10-12', '2017-09-16']
outg_rec_nbs = ['11947217', '11946640', '10143524']

sql_outage = (
"""
SELECT   DOV.CI_NB, DOV.CMI_NB, DOV.OUTG_REC_NB, DOV.OUTAGE_NB, DOV.DT_ON_TS, CAST(DOV.DT_OFF_TS AS DATE) AS DT_OFF_TS, 
         from_unixtime(unix_timestamp(DOV.DT_ON_TS)-60*DOV.STEP_DRTN_NB) AS DT_OFF_TS_FULL, 
         DOV.STEP_DRTN_NB,
         EXTRACT(YEAR FROM DOV.DT_OFF_TS)  AS START_YEAR, 
         DOV.OPERATING_UNIT_ID, DOV.STATE_ABBR_TX, 
         DOV.MJR_CAUSE_CD, DOV.MNR_CAUSE_CD,
         DOV1.OPRTG_UNT_NM, 
         DOV3.DVC_TYP_NM, DOV3.SHORT_NM AS SHORT_NM_CLR_DEV, 
         DOV4.EQUIP_TYP_NM, DOV4.SHORT_NM AS SHORT_NM_EQP_TYP, 
         DOV5.MJR_CAUSE_NM, DOV5.MNR_CAUSE_NM, 
         PRIM.OFF_TM, PRIM.REST_TM, PRIM.PREMISE_NB, 
         from_unixtime(unix_timestamp(DOV.DT_ON_TS)-60*DOV.STEP_DRTN_NB-1*60*60) AS t_search_min,
         from_unixtime(unix_timestamp(DOV.DT_ON_TS)+1*60*60) AS t_search_max
FROM     hdpsand.dovsadm_dovs_outage_fact DOV
         LEFT OUTER JOIN hdpsand.dovsadm_dovs_master_geo_dim DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
         LEFT OUTER JOIN hdpsand.dovsadm_dovs_outage_attributes_dim DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
         LEFT OUTER JOIN hdpsand.dovsadm_dovs_clearing_device_dim DOV3 ON DOV.DEVICE_CD=DOV3.DEVICE_CD
         LEFT OUTER JOIN hdpsand.dovsadm_dovs_equipment_types_dim DOV4 ON DOV.EQUIPMENT_CD=DOV4.EQUIPMENT_CD
         LEFT OUTER JOIN hdpsand.dovsadm_dovs_outage_cause_types_dim DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD
         LEFT OUTER JOIN hdpsand.dovsadm_dovs_premise_dim PRIM ON DOV.OUTG_REC_NB=PRIM.OUTG_REC_NB 
WHERE    CAST(DOV.DT_OFF_TS AS DATE) IN ({})
AND  DOV2.INTRPTN_TYP_CD = 'S'
AND  DOV2.CURR_REC_STAT_CD = 'A' 
AND  DOV.MJR_CAUSE_CD <> 'NI' 
AND  DOV.DEVICE_CD <> 85 
AND  DOV.STATE_ABBR_TX = 'OH'
AND  DOV.OUTG_REC_NB IN ({})
"""
).format(','.join(["'{}'".format(x) for x in dates_of_outages]), 
         ','.join(["'{}'".format(x) for x in outg_rec_nbs]))

print(sql_outage)

#--------------------
df_outage_2 = pd.read_sql(sql_outage, conn_aws)
df_outage_2 = Utilities_df.remove_prepend_from_columns_in_df(df_outage_2)
#--------------------
df_outage_2['dt_on_ts'] = pd.to_datetime(df_outage_2['dt_on_ts'])
df_outage_2['dt_off_ts'] = pd.to_datetime(df_outage_2['dt_off_ts'])
df_outage_2['dt_off_ts_full'] = pd.to_datetime(df_outage_2['dt_off_ts_full'])
df_outage_2['off_tm'] = pd.to_datetime(df_outage_2['off_tm'])
df_outage_2['rest_tm'] = pd.to_datetime(df_outage_2['rest_tm'])
df_outage_2['t_search_min'] = pd.to_datetime(df_outage_2['t_search_min'])
df_outage_2['t_search_max'] = pd.to_datetime(df_outage_2['t_search_max'])

#TODO
search_time_window = 60 #minutes
search_time_window = datetime.timedelta(minutes=search_time_window)

# NOTE: search_time_window can also be a vector of values...
df_outage_2['search_time_window'] = search_time_window
df_outage_2['t_search_min'] = df_outage_2['dt_off_ts_full'] - df_outage_2['search_time_window']
df_outage_2['t_search_max'] = df_outage_2['dt_on_ts'] + df_outage_2['search_time_window']

time_tot_half = time.time() - start
print(f'time_half = {time_tot_half}')
##############################################################################################
# df_end_dev_event_2
##############################################################################################
cols_of_interest_end_dev_event = [
    'issuertracking_id',    
    'serialnumber',
    'enddeviceeventtypeid',
    'valuesinterval',
    'aep_premise_nb',
    'reason',
    'event_type',
    'longitude',
    'latitude',
    'aep_opco',
    'aep_event_dt'    
]

cols_of_interest_met_prem = [
    'mfr_devc_ser_nbr',
    'longitude',
    'latitude',
    'state_cd', 
    'prem_nb',
    'srvc_pole_nb',
    'trsf_pole_nb',
    'latitude_nb',
    'longitude_nb',
    'station_nb', 
    'xfmr_nb'
]

unique_dates = sorted(set(df_outage_2['t_search_min'].dt.date.unique().tolist() + 
                          df_outage_2['t_search_max'].dt.date.unique().tolist()))

regexp_pattern = r'(\\d{4}-\\d{2}-\\d{2})T(\\d{2}:\\d{2}:\\d{2}).*'
partial_sql_ede = (
"""
WITH EDE as (
    SELECT {}
    FROM meter_events.end_device_event
    WHERE aep_opco = 'oh'
    AND aep_event_dt IN ({})
    AND (
        {{}}
    )
),"""
).format(','.join(cols_of_interest_end_dev_event),
         ','.join(["'{}'".format(x) for x in unique_dates]))
#-----
ede_datetime_ranges_string = ''
for i, (t_search_min_max, sub_df_outage_2) in enumerate(df_outage_2.groupby(['t_search_min', 't_search_max'])):
    str_to_fmt = "(aep_event_dt IN ({}) AND aep_premise_nb IN ({}) AND CAST(regexp_replace(valuesinterval, {}, '$1 $2') AS TIMESTAMP) BETWEEN {} AND {})"
    if i>0:
        str_to_fmt = "\n        OR"+str_to_fmt
    ede_datetime_ranges_string += str_to_fmt.format(
        ','.join(["'{}'".format(x) for x in sorted(set([x.date() for x in t_search_min_max]))]), 
        ','.join(["'{}'".format(x) for x in sub_df_outage_2['premise_nb'].unique().tolist()]), 
        f"'{regexp_pattern}'", 
        f"'{t_search_min_max[0]}'",
        f"'{t_search_min_max[1]}'"
    )   
partial_sql_ede = partial_sql_ede.format(ede_datetime_ranges_string)
#---------------------------------------------------------------------------------
partial_sql_mp = (
"""
MP AS (
    SELECT {}
    FROM default.meter_premise
    WHERE state_cd IN ({})
    AND prem_nb IN ({})
)
"""
).format(','.join(cols_of_interest_met_prem), 
         ','.join(["'{}'".format(x) for x in df_outage_2['state_abbr_tx'].unique().tolist()]), 
         ','.join(["'{}'".format(x) for x in df_outage_2['premise_nb'].unique().tolist()]))
#---------------------------------------------------------------------------------
sql_end_dev_event = (
"""{}{}
SELECT {}
FROM EDE
INNER JOIN MP
ON EDE.serialnumber = MP.mfr_devc_ser_nbr
"""
).format(partial_sql_ede, 
         partial_sql_mp, 
         ','.join([f'EDE.{x} AS {x}' for x in cols_of_interest_end_dev_event] + [f'MP.{x} AS {x}' for x in cols_of_interest_met_prem]))
#---------------------------------------------------------------------------------
print(sql_end_dev_event)

#--------------------
df_end_dev_event_2 = pd.read_sql(sql_end_dev_event, conn_aws)
df_end_dev_event_2 = Utilities_df.remove_prepend_from_columns_in_df(df_end_dev_event_2)
#--------------------
df_end_dev_event_2['valuesinterval_tz_aware'] = pd.to_datetime(df_end_dev_event_2['valuesinterval'])
df_end_dev_event_2['valuesinterval_local'] = df_end_dev_event_2['valuesinterval_tz_aware'].dt.tz_localize(None)
df_end_dev_event_2['valuesinterval_utc'] = df_end_dev_event_2['valuesinterval_tz_aware'].dt.tz_convert(None)
#--------------------
df_end_dev_event_2, n_events_with_mult_matches = clm.match_events_in_df_to_outages(df_end_dev_event_2, df_outage_2, 
                                                                                   lowercase_cols_default=lowercase_cols_default, 
                                                                                   return_n_events_with_mult_matches=True)
df_end_dev_event_2 = clm.set_all_outages_info_in_events_df(df_end_dev_event_2, df_outage_2, lowercase_cols_default=lowercase_cols_default)
#--------------------
time_tot_0 = time.time() - start
print(f'time_NEW = {time_tot_0}')

In [ ]:
df_end_dev_event_2

# ---------------------------------------------------------------
# USING SQL ONLY!
# ---------------------------------------------------------------

In [ ]:
dates_of_outages = ['2020-10-12', '2017-09-16']
outg_rec_nbs = ['11947217', '11946640', '10143524']
regexp_pattern = r'(\\d{4}-\\d{2}-\\d{2})T(\\d{2}:\\d{2}:\\d{2}).*'
sql_all_1 = (
"""
WITH DOVS_W_SNS
AS (
    SELECT   DOV.CI_NB, DOV.CMI_NB, DOV.OUTG_REC_NB, DOV.OUTAGE_NB, DOV.DT_ON_TS, DOV.DT_OFF_TS, 
             from_unixtime(unix_timestamp(DOV.DT_ON_TS)-60*DOV.STEP_DRTN_NB) AS DT_OFF_TS_FULL, 
             DOV.STEP_DRTN_NB,
             EXTRACT(YEAR FROM DOV.DT_OFF_TS) AS START_YEAR, 
             DOV.OPERATING_UNIT_ID, DOV.STATE_ABBR_TX, 
             DOV.MJR_CAUSE_CD, DOV.MNR_CAUSE_CD, 
             DOV.LOCATION_ID, DOV.GIS_CRCT_NB, 
             DOV1.OPRTG_UNT_NM, 
             DOV5.MJR_CAUSE_NM, DOV5.MNR_CAUSE_NM, 
             PRIM.OFF_TM, PRIM.REST_TM, PRIM.PREMISE_NB, 
             MP.mfr_devc_ser_nbr, MP.state_cd, MP.prem_nb, MP.trsf_pole_nb, 
             from_unixtime(unix_timestamp(from_unixtime(unix_timestamp(DOV.DT_ON_TS)-60*DOV.STEP_DRTN_NB))-1*60*60) AS t_search_min, 
             from_unixtime(unix_timestamp(DOV.DT_ON_TS)+1*60*60) AS t_search_max
    FROM     hdpsand.dovsadm_dovs_outage_fact DOV
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_master_geo_dim DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_outage_attributes_dim DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_outage_cause_types_dim DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_premise_dim PRIM ON DOV.OUTG_REC_NB=PRIM.OUTG_REC_NB 
             LEFT OUTER JOIN default.meter_premise MP ON PRIM.PREMISE_NB=MP.prem_nb AND DOV.STATE_ABBR_TX=MP.state_cd
    WHERE    CAST(DOV.DT_OFF_TS AS DATE) IN ({})
    AND  DOV2.INTRPTN_TYP_CD = 'S'
    AND  DOV2.CURR_REC_STAT_CD = 'A' 
    AND  DOV.MJR_CAUSE_CD <> 'NI' 
    AND  DOV.DEVICE_CD <> 85 
    AND  DOV.STATE_ABBR_TX = 'OH'
    AND  DOV.OUTG_REC_NB IN ({})
), 
EDE AS (
    SELECT issuertracking_id,serialnumber,enddeviceeventtypeid,valuesinterval,aep_premise_nb,reason,event_type,longitude,latitude,aep_opco,aep_event_dt
    FROM meter_events.end_device_event
    WHERE aep_opco = 'oh'
    AND (aep_event_dt IN (SELECT DISTINCT TO_DATE(DOVS_W_SNS.t_search_min) FROM DOVS_W_SNS)
      OR aep_event_dt IN (SELECT DISTINCT TO_DATE(DOVS_W_SNS.t_search_max) FROM DOVS_W_SNS)
    )
)

SELECT DOV.mfr_devc_ser_nbr, DOV.CI_NB, DOV.CMI_NB, DOV.OUTG_REC_NB, DOV.OUTAGE_NB, DOV.DT_ON_TS, EDE.serialnumber, EDE.enddeviceeventtypeid, EDE.valuesinterval, DOV.DT_OFF_TS_FULL
FROM DOVS_W_SNS DOV
INNER JOIN EDE
ON DOV.mfr_devc_ser_nbr = EDE.serialnumber AND CAST(regexp_replace(EDE.valuesinterval, {}, '$1 $2') AS TIMESTAMP) BETWEEN from_unixtime(unix_timestamp(DOV.DT_OFF_TS_FULL)-1*60*60) and from_unixtime(unix_timestamp(DOV.DT_ON_TS)+1*60*60)
"""
).format(','.join(["'{}'".format(x) for x in dates_of_outages]), 
         ','.join(["'{}'".format(x) for x in outg_rec_nbs]), 
         f"'{regexp_pattern}'")

print(sql_all_1)

In [ ]:
dates_of_outages = ['2020-10-12', '2017-09-16']
outg_rec_nbs = ['11947217', '11946640', '10143524']
regexp_pattern = r'(\\d{4}-\\d{2}-\\d{2})T(\\d{2}:\\d{2}:\\d{2}).*'
sql_all_2 = (
"""
WITH DOVS_W_SNs
AS (
    SELECT   DOV.CI_NB, DOV.CMI_NB, DOV.OUTG_REC_NB, DOV.OUTAGE_NB, DOV.DT_ON_TS, DOV.DT_OFF_TS, 
             from_unixtime(unix_timestamp(DOV.DT_ON_TS)-60*DOV.STEP_DRTN_NB) AS DT_OFF_TS_FULL, 
             DOV.STEP_DRTN_NB,
             EXTRACT(YEAR FROM DOV.DT_OFF_TS) AS START_YEAR, 
             DOV.OPERATING_UNIT_ID, DOV.STATE_ABBR_TX, 
             DOV.MJR_CAUSE_CD, DOV.MNR_CAUSE_CD, 
             DOV.LOCATION_ID, DOV.GIS_CRCT_NB, 
             DOV1.OPRTG_UNT_NM, 
             DOV5.MJR_CAUSE_NM, DOV5.MNR_CAUSE_NM, 
             PRIM.OFF_TM, PRIM.REST_TM, PRIM.PREMISE_NB, 
             MP.mfr_devc_ser_nbr, MP.state_cd, MP.prem_nb, MP.trsf_pole_nb,
             from_unixtime(unix_timestamp(DOV.DT_ON_TS)-60*DOV.STEP_DRTN_NB-1*60*60) AS t_search_min,
             from_unixtime(unix_timestamp(DOV.DT_ON_TS)+1*60*60) AS t_search_max
    FROM     hdpsand.dovsadm_dovs_outage_fact DOV
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_master_geo_dim DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_outage_attributes_dim DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_outage_cause_types_dim DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_premise_dim PRIM ON DOV.OUTG_REC_NB=PRIM.OUTG_REC_NB 
             LEFT OUTER JOIN default.meter_premise MP ON PRIM.PREMISE_NB=MP.prem_nb AND DOV.STATE_ABBR_TX=MP.state_cd
    WHERE    CAST(DOV.DT_OFF_TS AS DATE) IN ({})
    AND  DOV2.INTRPTN_TYP_CD = 'S'
    AND  DOV2.CURR_REC_STAT_CD = 'A' 
    AND  DOV.MJR_CAUSE_CD <> 'NI' 
    AND  DOV.DEVICE_CD <> 85 
    AND  DOV.STATE_ABBR_TX = 'OH'
    AND  DOV.OUTG_REC_NB IN ({})
), 
EDE AS (
    SELECT issuertracking_id,serialnumber,enddeviceeventtypeid,valuesinterval,aep_premise_nb,reason,event_type,longitude,latitude,aep_opco,aep_event_dt
    FROM meter_events.end_device_event
    WHERE aep_opco = 'oh'
    AND (aep_event_dt IN (SELECT DISTINCT TO_DATE(DOVS_W_SNS.t_search_min) FROM DOVS_W_SNS)
      OR aep_event_dt IN (SELECT DISTINCT TO_DATE(DOVS_W_SNS.t_search_max) FROM DOVS_W_SNS)
    )
)

SELECT DOV.mfr_devc_ser_nbr, DOV.CI_NB, DOV.CMI_NB, DOV.OUTG_REC_NB, DOV.OUTAGE_NB, DOV.DT_ON_TS, EDE.serialnumber, EDE.enddeviceeventtypeid, EDE.valuesinterval, DOV.DT_OFF_TS_FULL
FROM DOVS_W_SNS DOV
INNER JOIN EDE
ON DOV.mfr_devc_ser_nbr = EDE.serialnumber AND CAST(regexp_replace(EDE.valuesinterval, {}, '$1 $2') AS TIMESTAMP) BETWEEN from_unixtime(unix_timestamp(DOV.DT_OFF_TS_FULL)-1*60*60) and from_unixtime(unix_timestamp(DOV.DT_ON_TS)+1*60*60)
"""
).format(','.join(["'{}'".format(x) for x in dates_of_outages]), 
         ','.join(["'{}'".format(x) for x in outg_rec_nbs]), 
         f"'{regexp_pattern}'")

print(sql_all_2)

In [ ]:
start = time.time()
df_end_dev_event_NEW = pd.read_sql(sql_all_2, conn_aws)
time_tot_NEW = time.time() - start
print(f'time_NEW = {time_tot_NEW}')
df_end_dev_event_NEW = Utilities_df.remove_prepend_from_columns_in_df(df_end_dev_event_NEW)

In [ ]:
dates_of_outages = ['2020-10-12', '2017-09-16']
outg_rec_nbs = ['11947217', '11946640', '10143524']
regexp_pattern = r'(\\d{4}-\\d{2}-\\d{2})T(\\d{2}:\\d{2}:\\d{2}).*'
sql_all_3 = (
"""
WITH DOVS_W_SNS
AS (
    SELECT   DOV.CI_NB, DOV.CMI_NB, DOV.OUTG_REC_NB, DOV.OUTAGE_NB, DOV.DT_ON_TS, DOV.DT_OFF_TS, 
             from_unixtime(unix_timestamp(DOV.DT_ON_TS)-60*DOV.STEP_DRTN_NB) AS DT_OFF_TS_FULL, 
             DOV.STEP_DRTN_NB,
             EXTRACT(YEAR FROM DOV.DT_OFF_TS) AS START_YEAR, 
             DOV.OPERATING_UNIT_ID, DOV.STATE_ABBR_TX, 
             DOV.MJR_CAUSE_CD, DOV.MNR_CAUSE_CD, 
             DOV.LOCATION_ID, DOV.GIS_CRCT_NB, 
             DOV1.OPRTG_UNT_NM, 
             DOV5.MJR_CAUSE_NM, DOV5.MNR_CAUSE_NM, 
             PRIM.OFF_TM, PRIM.REST_TM, PRIM.PREMISE_NB, 
             MP.mfr_devc_ser_nbr, MP.state_cd, MP.prem_nb, MP.trsf_pole_nb, 
             from_unixtime(unix_timestamp(DOV.DT_ON_TS)-60*DOV.STEP_DRTN_NB-1*60*60) AS t_search_min,
             from_unixtime(unix_timestamp(DOV.DT_ON_TS)+1*60*60) AS t_search_max
    FROM     hdpsand.dovsadm_dovs_outage_fact DOV
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_master_geo_dim DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_outage_attributes_dim DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_outage_cause_types_dim DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_premise_dim PRIM ON DOV.OUTG_REC_NB=PRIM.OUTG_REC_NB 
             LEFT OUTER JOIN default.meter_premise MP ON PRIM.PREMISE_NB=MP.prem_nb AND DOV.STATE_ABBR_TX=MP.state_cd
    WHERE    CAST(DOV.DT_OFF_TS AS DATE) IN ({})
    AND  DOV2.INTRPTN_TYP_CD = 'S'
    AND  DOV2.CURR_REC_STAT_CD = 'A' 
    AND  DOV.MJR_CAUSE_CD <> 'NI' 
    AND  DOV.DEVICE_CD <> 85 
    AND  DOV.STATE_ABBR_TX = 'OH'
    AND  DOV.OUTG_REC_NB IN ({})
)

SELECT DOV.mfr_devc_ser_nbr, DOV.CI_NB, DOV.CMI_NB, DOV.OUTG_REC_NB, DOV.OUTAGE_NB, DOV.DT_ON_TS, EDE.serialnumber, EDE.enddeviceeventtypeid, EDE.valuesinterval, DOV.DT_OFF_TS_FULL
FROM DOVS_W_SNS DOV
INNER JOIN meter_events.end_device_event EDE
ON EDE.aep_event_dt BETWEEN TO_DATE(DOV.t_search_min) AND TO_DATE(DOV.t_search_max)
AND DOV.PREMISE_NB = EDE.aep_premise_nb
AND DOV.mfr_devc_ser_nbr = EDE.serialnumber 
AND CAST(regexp_replace(EDE.valuesinterval, {}, '$1 $2') AS TIMESTAMP) BETWEEN from_unixtime(unix_timestamp(DOV.DT_OFF_TS_FULL)-1*60*60) AND from_unixtime(unix_timestamp(DOV.DT_ON_TS)+1*60*60)
WHERE EDE.aep_opco = 'oh'
"""
).format(','.join(["'{}'".format(x) for x in dates_of_outages]), 
         ','.join(["'{}'".format(x) for x in outg_rec_nbs]), 
         f"'{regexp_pattern}'")

print(sql_all_3)

In [ ]:
start = time.time()
df_end_dev_event_NEW_3 = pd.read_sql(sql_all_3, conn_aws)
time_tot_NEW = time.time() - start
print(f'time_NEW = {time_tot_NEW}')

In [ ]:
dates_of_outages = ['2020-10-12', '2017-09-16']
outg_rec_nbs = ['11947217', '11946640', '10143524']
regexp_pattern = r'(\\d{4}-\\d{2}-\\d{2})T(\\d{2}:\\d{2}:\\d{2}).*'
sql_all_4 = (
"""
WITH DOVS_W_SNS
AS (
    SELECT   DOV.CI_NB, DOV.CMI_NB, DOV.OUTG_REC_NB, DOV.OUTAGE_NB, DOV.DT_ON_TS, DOV.DT_OFF_TS, 
             from_unixtime(unix_timestamp(DOV.DT_ON_TS)-60*DOV.STEP_DRTN_NB) AS DT_OFF_TS_FULL, 
             DOV.STEP_DRTN_NB,
             EXTRACT(YEAR FROM DOV.DT_OFF_TS) AS START_YEAR, 
             DOV.OPERATING_UNIT_ID, DOV.STATE_ABBR_TX, 
             DOV.MJR_CAUSE_CD, DOV.MNR_CAUSE_CD, 
             DOV.LOCATION_ID, DOV.GIS_CRCT_NB, 
             DOV1.OPRTG_UNT_NM, 
             DOV5.MJR_CAUSE_NM, DOV5.MNR_CAUSE_NM, 
             PRIM.OFF_TM, PRIM.REST_TM, PRIM.PREMISE_NB, 
             MP.mfr_devc_ser_nbr, MP.state_cd, MP.prem_nb, MP.trsf_pole_nb, 
             from_unixtime(unix_timestamp(DOV.DT_ON_TS)-60*DOV.STEP_DRTN_NB-1*60*60) AS t_search_min,
             from_unixtime(unix_timestamp(DOV.DT_ON_TS)+1*60*60) AS t_search_max
    FROM     hdpsand.dovsadm_dovs_outage_fact DOV
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_master_geo_dim DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_outage_attributes_dim DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_outage_cause_types_dim DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD
             LEFT OUTER JOIN hdpsand.dovsadm_dovs_premise_dim PRIM ON DOV.OUTG_REC_NB=PRIM.OUTG_REC_NB 
             LEFT OUTER JOIN default.meter_premise MP ON PRIM.PREMISE_NB=MP.prem_nb AND DOV.STATE_ABBR_TX=MP.state_cd
    WHERE    CAST(DOV.DT_OFF_TS AS DATE) IN ({})
    AND  DOV2.INTRPTN_TYP_CD = 'S'
    AND  DOV2.CURR_REC_STAT_CD = 'A' 
    AND  DOV.MJR_CAUSE_CD <> 'NI' 
    AND  DOV.DEVICE_CD <> 85 
    AND  DOV.STATE_ABBR_TX = 'OH'
    AND  DOV.OUTG_REC_NB IN ({})
), 
EDE AS (
    SELECT issuertracking_id,serialnumber,enddeviceeventtypeid,valuesinterval,aep_premise_nb,reason,event_type,longitude,latitude,aep_opco,aep_event_dt, 
           CAST(regexp_replace(valuesinterval, {}, '$1 $2') AS TIMESTAMP) AS valuesinterval_2
    FROM meter_events.end_device_event
    WHERE aep_opco = 'oh'
    AND (aep_event_dt IN (SELECT DISTINCT TO_DATE(DOVS_W_SNS.t_search_min) FROM DOVS_W_SNS)
      OR aep_event_dt IN (SELECT DISTINCT TO_DATE(DOVS_W_SNS.t_search_max) FROM DOVS_W_SNS)
    )
    AND aep_premise_nb IN (SELECT DISTINCT DOVS_W_SNS.PREMISE_NB FROM DOVS_W_SNS)
), 
MP AS (
    SELECT mfr_devc_ser_nbr,longitude,latitude,state_cd,prem_nb,srvc_pole_nb,trsf_pole_nb,latitude_nb,longitude_nb,station_nb,xfmr_nb
    FROM default.meter_premise
    WHERE state_cd IN ('OH')
    AND prem_nb IN (SELECT DISTINCT DOVS_W_SNS.PREMISE_NB FROM DOVS_W_SNS)
)

SELECT DOV.mfr_devc_ser_nbr, DOV.CI_NB, DOV.CMI_NB, DOV.OUTG_REC_NB, DOV.OUTAGE_NB, DOV.DT_ON_TS, EDE.serialnumber, EDE.enddeviceeventtypeid, EDE.valuesinterval, EDE.valuesinterval_2, DOV.DT_OFF_TS_FULL
FROM EDE
INNER JOIN MP ON EDE.serialnumber = MP.mfr_devc_ser_nbr
INNER JOIN DOVS_W_SNS DOV
ON DOV.mfr_devc_ser_nbr = EDE.serialnumber AND EDE.valuesinterval_2 BETWEEN DOV.t_search_min and DOV.t_search_max AND DOV.PREMISE_NB=MP.prem_nb AND DOV.STATE_ABBR_TX=MP.state_cd
"""
).format(','.join(["'{}'".format(x) for x in dates_of_outages]), 
         ','.join(["'{}'".format(x) for x in outg_rec_nbs]), 
         f"'{regexp_pattern}'")

print(sql_all_4)

In [ ]:
start = time.time()
df_end_dev_event_NEW_4 = pd.read_sql(sql_all_4, conn_aws)
time_tot_NEW = time.time() - start
print(f'time_NEW = {time_tot_NEW}')

In [ ]:
df_end_dev_event_NEW = df_end_dev_event_NEW.rename(columns={'outg_rec_nb':'OUTG_REC_NB', 'dt_on_ts':'DT_ON_TS', 'dt_off_ts_full':'DT_OFF_TS_FULL'})
chopped_new = df_end_dev_event_NEW[['OUTG_REC_NB', 'DT_ON_TS', 'DT_OFF_TS_FULL', 'serialnumber']].copy()
chopped_new = chopped_new.sort_values(by=['OUTG_REC_NB', 'serialnumber'], ignore_index=True)
chopped_new['DT_ON_TS'] = pd.to_datetime(chopped_new['DT_ON_TS'])
chopped_new['DT_OFF_TS_FULL'] = pd.to_datetime(chopped_new['DT_OFF_TS_FULL'])
chopped_new['OUTG_REC_NB'] = chopped_new['OUTG_REC_NB'].astype(int)
chopped_new

In [ ]:
chopped_old = df_end_dev_event[['OUTG_REC_NB', 'DT_ON_TS', 'DT_OFF_TS_FULL', 'serialnumber']].copy()
chopped_old = chopped_old.sort_values(by=['OUTG_REC_NB', 'serialnumber'], ignore_index=True)
chopped_old['OUTG_REC_NB'] = chopped_old['OUTG_REC_NB'].astype(int)
chopped_old

In [ ]:
chopped_old.equals(chopped_new)

In [ ]:
chopped_old.dtypes

In [ ]:
chopped_new.dtypes